In [1]:
# # Load the Drive helper and mount
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive')

In [2]:
!pip install mlxtend

In [3]:
# importing the libraries

import numpy as np
import pandas as pd
import os 

import matplotlib.pyplot as plt
import seaborn as sns

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

# Read CSV and Create empty dataframe 

In [4]:
# filepath = '/content/drive/MyDrive/DigitalCV_CourseRecommendation/'
filepath = './'


In [5]:
# reading the data
coursera_courses = pd.read_csv(f'{filepath}coursera_course/Coursera_courses.csv')
coursera_rating = pd.read_csv(f'{filepath}coursera_course/Coursera_reviews.csv')

### View Coursera Courses CSV file 

In [6]:
coursera_courses.head()

,name,institution,course_url,course_id
0,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning
1,Indigenous Canada,University of Alberta,https://www.coursera.org/learn/indigenous-canada,indigenous-canada
2,The Science of Well-Being,Yale University,https://www.coursera.org/learn/the-science-of-...,the-science-of-well-being
3,Technical Support Fundamentals,Google,https://www.coursera.org/learn/technical-suppo...,technical-support-fundamentals
4,Become a CBRS Certified Professional Installer...,Google - Spectrum Sharing,https://www.coursera.org/learn/google-cbrs-cpi...,google-cbrs-cpi-training


In [7]:
coursera_courses.dtypes

name           object
institution    object
course_url     object
course_id      object
dtype: object

In [8]:
coursera_courses.course_id.value_counts(),print('Number of duplicated unique ids are: ',coursera_courses.course_id.duplicated().sum())

Number of duplicated unique ids are:  0


(machine-learning                             1
 english-composition                          1
 wharton-communication-skills                 1
 introduction-trading-machine-learning-gcp    1
 python-programming-introduction              1
                                             ..
 solar-energy-basics                          1
 bootstrap-4                                  1
 google-cloud-java-spring                     1
 forensic-science                             1
 data-science-course                          1
 Name: course_id, Length: 623, dtype: int64,
 None)

### View Coursera Reviews CSV file 

In [9]:
coursera_rating.head()

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,"Feb 12, 2020",4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,"Sep 28, 2020",4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,"Apr 08, 2020",4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,By Dale B,"Feb 24, 2020",4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,By Sean G,"Jun 18, 2020",4,google-cbrs-cpi-training


In [10]:
coursera_rating.course_id.value_counts()

python                                        45218
machine-learning                              35895
neural-networks-deep-learning                 35750
technical-support-fundamentals                35319
python-data                                   33546
                                              ...  
social-economic-networks                          6
intro-to-healthcare                               6
entrepreneurship-strategy                         6
epigenetics                                       6
building-modern-python-applications-on-aws        3
Name: course_id, Length: 604, dtype: int64

## Merging Datasets

In [11]:
merge = coursera_courses.merge(coursera_rating,on = 'course_id',how = 'inner')

In [12]:
merge

,name,institution,course_url,course_id,reviews,reviewers,date_reviews,rating
0,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,This is an extremely basic course. Machine lea...,By Deleted A,"Mar 18, 2017",1
1,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,The course is ok but the certification procedu...,By Bruno C,"Nov 09, 2015",1
2,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,"I just started week 3 , I have to admit that I...",By Fadi,"Apr 15, 2019",1
3,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,This course is absolute garbage. You get no f...,By Mathew L,"Sep 25, 2015",1
4,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,"However good the material and lectures may be,...",By Rui C,"Dec 12, 2015",1
...,...,...,...,...,...,...,...,...
1454706,Managing as a Coach,"University of California, Davis",https://www.coursera.org/learn/managing-as-a-c...,managing-as-a-coach,re,By Sachin M,"Oct 11, 2017",5
1454707,Managing as a Coach,"University of California, Davis",https://www.coursera.org/learn/managing-as-a-c...,managing-as-a-coach,E,By Denis,"Mar 10, 2019",5
1454708,Managing as a Coach,"University of California, Davis",https://www.coursera.org/learn/managing-as-a-c...,managing-as-a-coach,G,By Neil B,"Oct 13, 2018",5
1454709,Managing as a Coach,"University of California, Davis",https://www.coursera.org/learn/managing-as-a-c...,managing-as-a-coach,E,By Isaias M,"Aug 11, 2016",5


Dropiong unneeded columns.

In [13]:
merge['reviewers'] = [f"{x.replace('By ', '')}" for x in merge['reviewers']]
merge.drop(columns=['institution','date_reviews','rating', 'reviews'],inplace=True)
merge.rename(columns={'name': 'CourseName',
                 'course_url': 'UrlLink',
                 'reviewers': 'UserId', 'course_id':'ourseId' }, inplace=True)


In [14]:
len(merge.UserId.unique()) 

287808

In [15]:
merge

,CourseName,UrlLink,ourseId,UserId
0,Machine Learning,https://www.coursera.org/learn/machine-learning,machine-learning,Deleted A
1,Machine Learning,https://www.coursera.org/learn/machine-learning,machine-learning,Bruno C
2,Machine Learning,https://www.coursera.org/learn/machine-learning,machine-learning,Fadi
3,Machine Learning,https://www.coursera.org/learn/machine-learning,machine-learning,Mathew L
4,Machine Learning,https://www.coursera.org/learn/machine-learning,machine-learning,Rui C
...,...,...,...,...
1454706,Managing as a Coach,https://www.coursera.org/learn/managing-as-a-c...,managing-as-a-coach,Sachin M
1454707,Managing as a Coach,https://www.coursera.org/learn/managing-as-a-c...,managing-as-a-coach,Denis
1454708,Managing as a Coach,https://www.coursera.org/learn/managing-as-a-c...,managing-as-a-coach,Neil B
1454709,Managing as a Coach,https://www.coursera.org/learn/managing-as-a-c...,managing-as-a-coach,Isaias M


In [16]:
merge_list = merge.groupby(by = ["UserId"])["CourseName"].apply(list).reset_index()
merge_list.head()

,UserId,CourseName
0,\t M N H,"[Grammar and Punctuation, Grammar and Punctuat..."
1,\t M R I,"[Python Data Structures, Python Data Structure..."
2,\t M S R,"[Guitar for Beginners, Guitar for Beginners, G..."
3,\t N A F B,[Digital Product Management: Modern Fundamenta...
4,\t N H P,"[Speak English Professionally: In Person, Onli..."


In [17]:
merge_list = merge_list["CourseName"].tolist()
merge_list[0:3]

[['Grammar and Punctuation',
  'Grammar and Punctuation',
  'Grammar and Punctuation'],
 ['Python Data Structures',
  'Python Data Structures',
  'Python Data Structures'],
 ['Guitar for Beginners', 'Guitar for Beginners', 'Guitar for Beginners']]

In [18]:
len(merge_list)

287808

In [19]:
## DATA TRANSFORMATION

In [20]:
te = TransactionEncoder()
te_ary = te.fit(merge_list).transform(merge_list)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [21]:
df.head()

,A Crash Course in Causality: Inferring Causal Effects from Observational Data,A Law Student's Toolkit,AI For Everyone,AI for Medical Diagnosis,AI for Medical Prognosis,AWS Fundamentals: Addressing Security Risk,AWS Fundamentals: Going Cloud-Native,Aboriginal Worldviews and Education,Access Controls,Accounting Analytics,...,Write Professional Emails in English,Write Your First Novel,Writing and Editing: Word Choice and Word Order,Writing in the Sciences,e-Learning Ecologies: Innovative Approaches to Teaching and Learning for the Digital Age,Étudier en France: French Intermediate course B1-B2,Математика и Python для анализа данных,Основы программирования на Python,Основы разработки на C++: белый пояс,Разработка веб-сервисов на Go - основы языка
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## GENERATING FREQUENT ITEMSETS

### APRIORI

In [22]:
apriori_frequent_itemsets = apriori(df, min_support=0.01,use_colnames=True,max_len=2)

In [23]:
apriori_frequent_itemsets['itemsets'].apply(lambda x: len(x)).value_counts()

1    32
2     1
Name: itemsets, dtype: int64

### Fpgrowth

In [24]:
%time
fpgrowth_frequent_itemsets = fpgrowth(df, min_support=0.0001, use_colnames=True,max_len=5)
fpgrowth_frequent_itemsets.head()

CPU times: total: 0 ns
Wall time: 0 ns


,support,itemsets
0,0.015364,(Grammar and Punctuation)
1,0.037254,(Python Data Structures)
2,0.001835,(Guitar for Beginners)
3,0.002550,(Digital Product Management: Modern Fundamentals)
4,0.007543,"(Speak English Professionally: In Person, Onli..."


In [25]:
fpgrowth_frequent_itemsets['itemsets'].apply(lambda x: len(x)).value_counts()

4    23925
5    19506
3    17268
2     7610
1      577
Name: itemsets, dtype: int64

### ANAYSIS TO THE RESULTS

In [26]:
fpgrowth_frequent_itemsets['length'] = fpgrowth_frequent_itemsets['itemsets'].apply(lambda x: len(x))
fpgrowth_frequent_itemsets

,support,itemsets,length
0,0.015364,(Grammar and Punctuation),1
1,0.037254,(Python Data Structures),1
2,0.001835,(Guitar for Beginners),1
3,0.002550,(Digital Product Management: Modern Fundamentals),1
4,0.007543,"(Speak English Professionally: In Person, Onli...",1
...,...,...,...
68881,0.000129,(Learning How to Learn: Powerful mental tools ...,2
68882,0.000115,"(Machine Learning, Improving your statistical ...",2
68883,0.000101,"(Technical Support Fundamentals, Improving you...",2
68884,0.000108,(Learning How to Learn: Powerful mental tools ...,2


In [27]:
fpgrowth_frequent_itemsets[(fpgrowth_frequent_itemsets['length'] > 1)
                          & (fpgrowth_frequent_itemsets['support'] > 0.06)].head()

,support,itemsets,length


In [28]:
fpgrowth_frequent_itemsets[(fpgrowth_frequent_itemsets['length'] != 1)]

,support,itemsets,length
577,0.000413,"(Grammar and Punctuation, Google Cloud Platfor...",2
578,0.000632,"(Grammar and Punctuation, AI For Everyone)",2
579,0.001119,"(Grammar and Punctuation, Programming for Ever...",2
580,0.000441,"(Grammar and Punctuation, Stanford Introductio...",2
581,0.000900,"(Python Data Structures, Grammar and Punctuation)",2
...,...,...,...
68881,0.000129,(Learning How to Learn: Powerful mental tools ...,2
68882,0.000115,"(Machine Learning, Improving your statistical ...",2
68883,0.000101,"(Technical Support Fundamentals, Improving you...",2
68884,0.000108,(Learning How to Learn: Powerful mental tools ...,2


## ASSOCIATION RULES

In [29]:
%time
rules = association_rules(fpgrowth_frequent_itemsets,metric="lift",min_threshold=0.01)
rules

CPU times: total: 0 ns
Wall time: 0 ns


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Grammar and Punctuation),(Google Cloud Platform Fundamentals: Core Infr...,0.015364,0.019659,0.000413,0.026911,1.368889,0.000111,1.007453
1,(Google Cloud Platform Fundamentals: Core Infr...,(Grammar and Punctuation),0.019659,0.015364,0.000413,0.021032,1.368889,0.000111,1.005790
2,(Grammar and Punctuation),(AI For Everyone),0.015364,0.024673,0.000632,0.041158,1.668153,0.000253,1.017193
3,(AI For Everyone),(Grammar and Punctuation),0.024673,0.015364,0.000632,0.025630,1.668153,0.000253,1.010536
4,(Grammar and Punctuation),(Programming for Everybody (Getting Started wi...,0.015364,0.048862,0.001119,0.072818,1.490260,0.000368,1.025837
...,...,...,...,...,...,...,...,...,...
1038953,(Improving your statistical inferences),(Technical Support Fundamentals),0.000678,0.037883,0.000101,0.148718,3.925728,0.000075,1.130198
1038954,(Learning How to Learn: Powerful mental tools ...,(Improving your statistical inferences),0.038387,0.000678,0.000108,0.002806,4.141391,0.000082,1.002134
1038955,(Improving your statistical inferences),(Learning How to Learn: Powerful mental tools ...,0.000678,0.038387,0.000108,0.158974,4.141391,0.000082,1.143382
1038956,(Neural Networks and Deep Learning),(Improving your statistical inferences),0.039367,0.000678,0.000101,0.002560,3.777777,0.000074,1.001887


## Export rules to pickle file 

In [43]:
# Save rule to pickle 
rules.to_pickle('./pickle_folder/rules.pkl')

## View 

In [37]:
df1 = rules[rules["antecedents"].apply(lambda x: "AI For Everyone" in str(x))].sort_values(ascending=False,by='lift')
df2 = df1[df1["antecedents"].apply(lambda x: "Neural Networks and Deep Learning" in str(x))].sort_values(ascending=False,by='lift')
df2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
418542,"(The Bits and Bytes of Computer Networking, Ne...",(Operating Systems and You: Becoming a Power U...,0.000351,0.000479,0.000111,0.316832,660.773138,0.000111,1.463066
358821,"(Excel Skills for Business: Essentials, Neural...","(Introduction to Programming with MATLAB, Intr...",0.000507,0.000302,0.000101,0.198630,657.095890,0.000101,1.247486
358819,"(Introduction to Programming with MATLAB, Neur...","(Excel Skills for Business: Essentials, Introd...",0.000285,0.000570,0.000101,0.353659,620.644854,0.000101,1.546288
896546,"(Data Science Methodology, Neural Networks and...","(Tools for Data Science, Machine Learning)",0.000188,0.001094,0.000115,0.611111,558.357672,0.000114,2.568614
418546,(Operating Systems and You: Becoming a Power U...,"(The Bits and Bytes of Computer Networking, Im...",0.000240,0.000883,0.000111,0.463768,525.496748,0.000111,1.863219
...,...,...,...,...,...,...,...,...,...
289724,"(Neural Networks and Deep Learning, AI For Eve...",(Psychological First Aid),0.002189,0.009979,0.000136,0.061905,6.203581,0.000114,1.055352
13480,"(Neural Networks and Deep Learning, AI For Eve...",(Stanford Introduction to Food and Health),0.002189,0.016719,0.000202,0.092063,5.506361,0.000165,1.082984
170978,"(Neural Networks and Deep Learning, AI For Eve...",(English for Career Development),0.002189,0.010114,0.000122,0.055556,5.492729,0.000099,1.048114
27112,"(Neural Networks and Deep Learning, AI For Eve...",(The Science of Well-Being),0.002189,0.026302,0.000316,0.144444,5.491713,0.000259,1.138088


In [35]:
df2["consequents"] = df2["consequents"].apply(lambda x: list(x)[0]).astype("unicode")

In [36]:
df2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
358821,"(Excel Skills for Business: Essentials, Neural...",Introduction to Programming with MATLAB,0.000507,0.000302,0.000101,0.198630,657.095890,0.000101,1.247486
358788,"(Excel Skills for Business: Essentials, Progra...",Introduction to Programming with MATLAB,0.000566,0.000302,0.000108,0.190184,629.155067,0.000108,1.234475
358793,"(Introduction to Programming with MATLAB, AI F...",Programming for Everybody (Getting Started wit...,0.000188,0.001070,0.000108,0.574074,536.438672,0.000108,2.345314
336220,"(Excel Skills for Business: Essentials, The Da...",Technical Support Fundamentals,0.000285,0.000799,0.000101,0.353659,442.546766,0.000101,1.545933
358820,"(Excel Skills for Business: Essentials, AI For...",Introduction to Programming with MATLAB,0.000236,0.000994,0.000101,0.426471,429.166598,0.000101,1.741857
...,...,...,...,...,...,...,...,...,...
711962,"(Excel Skills for Business: Essentials, AI For...",Introduction to Psychology,0.001268,0.014266,0.000156,0.123288,8.641787,0.000138,1.124352
25783,"(Excel Skills for Business: Essentials, AI For...",Technical Support Fundamentals,0.001268,0.037883,0.000413,0.326027,8.606190,0.000365,1.427531
13432,"(Excel Skills for Business: Essentials, AI For...",Stanford Introduction to Food and Health,0.001268,0.016719,0.000163,0.128767,7.701623,0.000142,1.128608
25813,"(Excel Skills for Business: Essentials, AI For...",Learning How to Learn: Powerful mental tools t...,0.001268,0.038387,0.000361,0.284932,7.422662,0.000313,1.344785


In [32]:
rules[rules["antecedents"].apply(lambda x: "AI For Everyone" in str(x))].groupby(
    ['antecedents', 'consequents'])[['lift']].max().sort_values(ascending=False,by='lift')

lift
antecedents                                        consequents                                                   
(The Bits and Bytes of Computer Networking, AI ... (Operating Systems and You: Becoming a Power Us...  673.463768
(Operating Systems and You: Becoming a Power Us... (The Bits and Bytes of Computer Networking, R P...  672.448598
(The Bits and Bytes of Computer Networking, Neu... (Operating Systems and You: Becoming a Power Us...  660.773138
(The Bits and Bytes of Computer Networking, AI ... (Operating Systems and You: Becoming a Power Us...  660.528015
(Excel Skills for Business: Essentials, Neural ... (Introduction to Programming with MATLAB, Intro...  657.095890
...                                                                                                           ...
(AI For Everyone)                                  (Teach English Now! Foundational Principles)          0.861372
                                                   (Child Nutrition and Cooking)                         0.797770
                                                   (Introducción a Data Science: Programación Esta...    0.743523
                                                   (Aprendiendo a aprender: Poderosas herramientas...    0.604333
                                                   (First Step Korean)                                   0.523954

[56183 rows x 1 columns]